### Steps to be done:

### Install necessary libraries

In [ ]:
from google.colab import files
import pkg_resources

required_pkgs=["gensim","vaderSentiment"]
installed_packages = pkg_resources.working_set
installed_package_names = list(map(lambda item : item.project_name, installed_packages))

if any(pkg in required_pkgs for pkg in installed_package_names) :
  print("""
  -------------------------------
  All packages installed!
  -------------------------------
  """)
else :
  print("""
  /////////////////////////////////////////////
  Please upload supplied requirements.txt file.
  /////////////////////////////////////////////
  """)
  files.upload()
  print("""
  ==================================================================================================
  NOTE: You will need to restart the session anc click "Run All" again after all packeges download.
  ==================================================================================================
  """)

/tmp/ipython-input-3668611086.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources



  /////////////////////////////////////////////
  Please upload supplied requirements.txt file.
  /////////////////////////////////////////////
  


Saving requirements.txt to requirements.txt

  NOTE: You will need to restart the session anc click "Run All" again after all packeges download.
  


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 10.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.1 requires scipy>=1.14.0; python_ver

### NOTE as you will see later, the data we will use has a lot of duplicates which will lead to problems down the road with the LSTM. You may run into an overfitting issue (and sometimes underfitting) and is therefore the least stable model for this task with the data given.

(Please see the second cell for EDA for more information)

### First, I am going to import all of the necessary libraries into the project

In [ ]:
from tensorflow.keras import layers, Sequential
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from transformers import pipeline
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import F1Score
from zipfile import ZipFile
from gensim import corpora, models
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad, AdamW
from tensorflow.keras.utils import Sequence
from google.colab import drive
import os
import gdown
import cv2
import random
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import string
import re
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

os.environ["WANDB_DISABLED"] = "true"

#Check if the drive is present just in case
drive_path = '/content/drive/MyDrive/'
if os.path.exists(drive_path):
    drive.flush_and_unmount()
else:
    print("Drive not found.")

#Next, extract the csv file from googe drive
if os.path.exists("bike_rental_reviews.csv") == False :
  url = "https://drive.google.com/uc?id=1WqI-jkiUR9SjBcTwx7kRgPZzpBrplxkB"
  output = "bike_rental_reviews.csv"
  gdown.download(url, output)

reviews_df = pd.read_csv("bike_rental_reviews.csv")

Drive not found.


Downloading...
From: https://drive.google.com/uc?id=1WqI-jkiUR9SjBcTwx7kRgPZzpBrplxkB
To: /content/bike_rental_reviews.csv
100%|██████████| 3.56M/3.56M [00:00<00:00, 103MB/s]


### EDA

In [ ]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_text  50000 non-null  object
 1   sentiment    50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
print("TOTAL DUPLICATE REVIEWS => " + str(reviews_df.duplicated().sum()))
reviews_df[reviews_df.duplicated()]

TOTAL DUPLICATE REVIEWS => 49700


,review_text,sentiment
7,Worst experience ever. The pricing ruined the ...,negative
33,A decent ride but not the best customer service.,neutral
42,Too expensive for such poor quality mobile app.,negative
44,Long wait times and the customer service was a...,negative
51,Worst experience ever. The return process ruin...,negative
...,...,...
49995,Bikes were in terrible condition. The mobile a...,negative
49996,Standard rental process. The availability was ...,neutral
49997,Absolutely loved the mobile app! The experienc...,positive
49998,Not worth the money. The rental station was a ...,negative


In [ ]:
#remove duplicates
reviews_df = reviews_df.drop_duplicates().reset_index(drop=True)
reviews_df

,review_text,sentiment
0,"The entire process was easy, and the availabil...",positive
1,Standard rental process. The mobile app was ac...,neutral
2,One of the best bike rentals I’ve had. The mob...,positive
3,One of the best bike rentals I’ve had. The cus...,positive
4,Not worth the money. The seat comfort was a ma...,negative
...,...,...
295,Nothing extraordinary. The bike condition was ...,neutral
296,"Nothing to complain about, but the mobile app ...",neutral
297,"The mobile app was well-maintained, and the re...",positive
298,Too expensive for such poor quality support st...,negative


### Next I am going to perform the necessary operations of combining the sentiment csv reviews with the reviews from the review database to form proper x and y training sets.

### Now, I am going to prepare the data according to what is needed of the various models.

#### First, I am going to tokenize the reviews.

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

reviews = list(reviews_df["review_text"].values.astype(str))

tokenized_reviews = []

for review in reviews:
  new_review = nltk.word_tokenize(review)
  tokenized_reviews.append(new_review)

print(tokenized_reviews)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


[['The', 'entire', 'process', 'was', 'easy', ',', 'and', 'the', 'availability', 'was', 'high', 'quality', '.'], ['Standard', 'rental', 'process', '.', 'The', 'mobile', 'app', 'was', 'acceptable', '.'], ['One', 'of', 'the', 'best', 'bike', 'rentals', 'I', '’', 've', 'had', '.', 'The', 'mobile', 'app', 'made', 'it', 'even', 'better', '.'], ['One', 'of', 'the', 'best', 'bike', 'rentals', 'I', '’', 've', 'had', '.', 'The', 'customer', 'service', 'made', 'it', 'even', 'better', '.'], ['Not', 'worth', 'the', 'money', '.', 'The', 'seat', 'comfort', 'was', 'a', 'major', 'letdown', '.'], ['Worst', 'experience', 'ever', '.', 'The', 'pricing', 'ruined', 'the', 'entire', 'ride', '.'], ['It', 'was', 'fine', ',', 'but', 'I', 'wouldn', '’', 't', 'rave', 'about', 'the', 'return', 'process', '.'], ['A', 'decent', 'ride', 'but', 'not', 'the', 'best', 'customer', 'service', '.'], ['Top-notch', 'service', 'with', 'great', 'pricing', '.'], ['An', 'average', 'experience', '.', 'The', 'availability', 'worked

#### Next I am going to lemmatize each word, remove the punctuation, stop words, make every word lower case, and lemmetize each word.

In [ ]:
#Remove token words
removed_token_words = []
for words in tokenized_reviews:
  label_words = set(reviews_df["sentiment"].values)
  new_words = []
  for word in words :
    if word.lower() not in label_words:
      new_words.append(word)
  removed_token_words.append(new_words)

#lemmatize words
lemmatized_words = []
lemmatizer = nltk.stem.WordNetLemmatizer()
for tok_review in removed_token_words :
  new_lematized_array = []

  for word in tok_review:
    lemmatized_word = lemmatizer.lemmatize(word)
    new_lematized_array.append(lemmatized_word)

  lemmatized_words.append(new_lematized_array)

#make words lower case
lower_case_reviews = []
for tok_review in tokenized_reviews :
  tokenized_review = list(map(str.lower, tok_review))
  lower_case_reviews.append(tokenized_review)

# #remove punctuation
no_punctuation_reviews = []
for tok_review in lower_case_reviews:
  new_review = [rev for rev in tok_review if rev not in string.punctuation]
  no_punctuation_reviews.append(new_review)

#remove stop words
no_stop_words_reviews = []
for tok_review in no_punctuation_reviews:
  stop_words = nltk.corpus.stopwords.words('english')
  new_review = [rev for rev in tok_review if rev not in stop_words]
  no_stop_words_reviews.append(new_review)

#fix contractions
fixed_contractions_reviews = []
for tok_review in no_stop_words_reviews:

  review_string = " ".join(tok_review)
  contraction_regex = r"(\s[a-zA-z]*\s)(’)(\s[A-Za-z]*\s)"
  contraction_string_match = re.search(contraction_regex, review_string)

  if contraction_string_match :
    new_string = ""
    match_start = 0
    match_end = 0

    for matched in range(1, len(contraction_string_match.groups())+1) :
      match_value = "".join(contraction_string_match.groups(matched))
      new_string = "".join(match_value).replace(" ", "")
      review_string = review_string.replace(match_value, " "+new_string+" ")

  fixed_contractions_reviews.append(review_string.split(" "))

final_text_token_output = fixed_contractions_reviews
print(len(final_text_token_output))

300


### Now to remove the old reviews and then combine them with the new reviews in the reviews_df dataframe

In [ ]:
def combine_tokens_to_string(tokens):
  combined_tokens = []
  for token in tokens :
    combined_tokens.append(" ".join(token))
  return combined_tokens

new_reviews = pd.DataFrame({
    "review_text":combine_tokens_to_string(final_text_token_output),
    "sentiment":reviews_df["sentiment"]
})

reviews_df = new_reviews

### I will now use LDA topic modeling in order to extract key topics on the model

In [ ]:
topic_dictionary = corpora.Dictionary(final_text_token_output)
new_corpus=[topic_dictionary.doc2bow(token) for token in final_text_token_output]
lda_topic_builder = models.LdaModel(
    id2word=topic_dictionary,
    corpus=new_corpus,
    num_topics=7,
    passes=10,
    random_state=42
)
generated_topics=lda_topic_builder.print_topics(num_words=5)

for topic in generated_topics:
  print(topic)

def get_current_topics(token_array):
  constructed_topics = []
  final_topics = []

  for tokens in token_array :
    tokens_array = topic_dictionary.doc2bow(tokens)
    current_topics =  lda_topic_builder.get_document_topics(tokens_array)
    if current_topics :
      constructed_topics.append(current_topics)
    else :
      constructed_topics.append(None)

  for topic in constructed_topics:
    final_topics.append(str(max(topic, key=lambda item : item[1])[0]))

  return final_topics

topics = get_current_topics(final_text_token_output)

(0, '0.118*"process" + 0.047*"comfort" + 0.047*"seat" + 0.046*"return" + 0.034*"quality"')
(1, '0.097*"service" + 0.069*"poor" + 0.043*"customer" + 0.039*"nothing" + 0.035*"long"')
(2, '0.117*"experience" + 0.057*"decent" + 0.057*"ride" + 0.044*"rental" + 0.030*"ever"')
(3, '0.106*"bike" + 0.067*"best" + 0.064*"one" + 0.064*"made" + 0.064*"even"')
(4, '0.082*"rental" + 0.068*"station" + 0.050*"nothing" + 0.040*"support" + 0.040*"bikes"')
(5, '0.101*"battery" + 0.101*"life" + 0.074*"process" + 0.059*"fine" + 0.052*"everything"')
(6, '0.106*"bikes" + 0.050*"condition" + 0.049*"rental" + 0.047*"process" + 0.040*"app"')


#### Here are the dominant topics that customers often discuss:
---
| TOPIC | WORDS |
|---|:---:|
| Bike Comfort Quality | process, comfort, seat, return, quality |
| Customer Service | service, poor, customer, nothing, long |
| Ride Experience | experience, decent, ride, rental, ever |
| Bike Overall Quality | bike, best, one, made, even |
| Rental Experience | rental, station, nothing, support, bikes |
| Bike Battery Life | battery, life, process, fine, everything |
| Bike Condition | bikes, condition, rental, process, app |

### Now I am going to train a logistic regression and a naive bays model on the new dataset.



#### train test split and vectorization

### NOTE: I was thinking about using word2vec for vectorization, but since my corpus only has 299 rows after removing duplicates, and word2vec requires A LOT of text data to make meaningful vector calculations for context, etc. TF-IDFVectorizer would be the best way to get basic vectorized text in this case

In [ ]:
X = reviews_df["review_text"]
y = reviews_df["sentiment"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

tfidf = TfidfVectorizer(max_features=80,
    min_df=2,
    max_df=0.3,
    stop_words='english'
)

X_train_vectorized = tfidf.fit_transform(X_train.tolist())
X_test_vectorized = tfidf.transform(X_test.tolist())
X_train

,review_text
118,frustrating process bike condition working pro...
210,decent ride best bike condition
142,one best bike rentals’support staff made even ...
254,bikes decent customer service okay
282,nothing complain rental station’great either
...,...
206,bikes decent support staff okay
66,everything booking ride flawless loved support...
184,everything fine rental station could improved
8,top-notch service great pricing


#### Here is the LogisticRegression model

In [ ]:
lr_model = LogisticRegression(max_iter=1000)

grid = GridSearchCV(
    estimator=lr_model,
    param_grid={},
    cv=5,
)

grid.fit(X_train_vectorized, y_train)
best_model = grid.best_estimator_
lr_y_pred = best_model.predict(X_test_vectorized)

report = classification_report(y_test, lr_y_pred)
print(report)
cm = confusion_matrix(y_test, lr_y_pred)
print("CONFUSION MATRIX : \n", cm)

              precision    recall  f1-score   support

    negative       1.00      0.85      0.92        20
     neutral       0.95      1.00      0.98        20
    positive       0.91      1.00      0.95        20

    accuracy                           0.95        60
   macro avg       0.95      0.95      0.95        60
weighted avg       0.95      0.95      0.95        60

CONFUSION MATRIX : 
 [[17  1  2]
 [ 0 20  0]
 [ 0  0 20]]


### Here is the Naive-Bayes Model

In [ ]:
nb_model = MultinomialNB(alpha=0.5)
grid2 = GridSearchCV(
    estimator=nb_model,
    param_grid={},
    cv=5,
)

grid2.fit(X_train_vectorized, y_train)
best_model2 = grid2.best_estimator_
nb_y_pred = best_model2.predict(X_test_vectorized)

report = classification_report(y_test, nb_y_pred)
print(report)
cm = confusion_matrix(y_test, nb_y_pred)
print("CONFUSION MATRIX : \n", cm)

              precision    recall  f1-score   support

    negative       1.00      0.80      0.89        20
     neutral       0.87      1.00      0.93        20
    positive       0.90      0.95      0.93        20

    accuracy                           0.92        60
   macro avg       0.92      0.92      0.92        60
weighted avg       0.92      0.92      0.92        60

CONFUSION MATRIX : 
 [[16  2  2]
 [ 0 20  0]
 [ 0  1 19]]


#### In conclusion, the logistic regression model overfitted, while the naive-bayes model gave us the best results, of which, is expected behavior.

### Next, I am going to use an LSTM and also a transformer from hugging face (VADER) and see how thses perform with the data at hand.

In [ ]:
vocab_size = 3000
embedding_dim = 64
max_length=200
padding_type = 'post'
trunc_type = 'post'

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(y_train)
test_labels_encoded = label_encoder.fit_transform(y_test)
print(len(y_test))
train_reviews = list(X_train.values)
test_reviews = list(X_test.values)

tokenizer = Tokenizer(num_words = 3000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_reviews)

train_seq = tokenizer.texts_to_sequences(train_reviews)
test_seq = tokenizer.texts_to_sequences(test_reviews)
train_padded = pad_sequences(train_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
test_padded = pad_sequences(test_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
train_tokens = np.array(train_padded)
test_tokens = np.array(test_padded)

early_stop = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

model_lstm = Sequential([
    layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    layers.Bidirectional(layers.LSTM(12, dropout=0.75, recurrent_dropout=0.75)),
    layers.Dense(3, activation='softmax')
])

model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history_lstm = model_lstm.fit(train_tokens, train_labels_encoded, epochs=8, validation_data=(test_tokens, test_labels_encoded),callbacks=[early_stop], shuffle=True)
model_probs = model_lstm.predict(test_tokens)
y_pred_labels = np.argmax(model_probs, axis=1)

# model_lstm.save('sentiment_lstm_v1.0.keras')

print(classification_report(test_labels_encoded, y_pred_labels, target_names=['negative', 'neutral', 'positive']))
cm = confusion_matrix(test_labels_encoded, y_pred_labels)
print("CONFUSION MATRIX : \n",cm)

60
Epoch 1/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 23s 721ms/step - accuracy: 0.3183 - loss: 1.1546 - val_accuracy: 0.3167 - val_loss: 1.0956
Epoch 2/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.3850 - loss: 1.1058 - val_accuracy: 0.3333 - val_loss: 1.0873
Epoch 3/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 317ms/step - accuracy: 0.3844 - loss: 1.1059 - val_accuracy: 0.5167 - val_loss: 1.0790
Epoch 4/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 438ms/step - accuracy: 0.4042 - loss: 1.0835 - val_accuracy: 0.6000 - val_loss: 1.0701
Epoch 5/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 286ms/step - accuracy: 0.3841 - loss: 1.0963 - val_accuracy: 0.9333 - val_loss: 1.0611
Epoch 6/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 285ms/step - accuracy: 0.4557 - loss: 1.0749 - val_accuracy: 0.9333 - val_loss: 1.0515
Epoch 7/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 294ms/step - accuracy: 0.4757 - loss: 1.0746 - val_accuracy: 0.9500 - val_loss: 1.0406
Epoch 8/8
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 427ms/step - accuracy: 0.5597 - loss: 1.0311 - val_accuracy: 0.9167 - val_loss: 1.0296

### Now for a transformer from huggingface (BERT)

In [ ]:
training_folder_path = '/content/training'

if os.path.isdir(training_folder_path) == False :
  os.mkdir(training_folder_path)
else :
  print("Folder already exists.")

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
bert_model.dropout.p = 0.3

reviews_train_data = pd.DataFrame({
    "review_text":X_train.tolist(),
    "sentiment":y_train.tolist()
})

reviews_test_data = pd.DataFrame({
    "review_text":X_test.tolist(),
    "sentiment":y_test.tolist()
})

train_dataset = Dataset.from_pandas(reviews_train_data)
test_dataset = Dataset.from_pandas(reviews_test_data)

def add_labels(example):
    label_map = {"positive":0, "neutral":1, "negative":2}
    example["labels"] = label_map[example["sentiment"]]
    return example

token_train_data = train_dataset.map(lambda batch : tokenizer(batch["review_text"], padding=True, truncation=True), batched=True)
token_test_data = test_dataset.map(lambda batch : tokenizer(batch["review_text"], padding=True, truncation=True), batched=True)
final_train_data = token_train_data.map(add_labels)
final_test_data = token_test_data.map(add_labels)

training_args = TrainingArguments(
    output_dir="./training",
    eval_strategy="epoch",
    num_train_epochs=2,
    label_smoothing_factor=0.1,
    weight_decay=0.01
)

trainer_obj = Trainer(model=bert_model, args=training_args, train_dataset=final_train_data, eval_dataset=final_test_data)
trainer_obj.train()

y_true_keys = {0:"positive", 1:"neutral", 2:"negative"}
y_pred = trainer_obj.predict(final_test_data)
y_preds = np.argmax(y_pred.predictions, axis=1)
y_labels = [y_true_keys[pred] for pred in y_preds]
y_true = list(reviews_test_data["sentiment"])

print(y_pred.predictions[:5])

print(classification_report(y_true, y_labels, target_names=['negative', 'neutral', 'positive']))
cm = confusion_matrix(y_true, y_labels)
print(cm)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,No log,0.640977
2,No log,0.517413


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[[ 1.3512686  -1.0974002   1.0594265 ]
 [ 1.0566134  -0.94066966  1.5028405 ]
 [ 0.7951731  -0.88746655 -0.86031294]
 [ 1.5822675  -0.8688397   0.3694378 ]
 [ 1.5797225  -0.9544627   0.67111033]]
              precision    recall  f1-score   support

    negative       1.00      0.90      0.95        20
     neutral       1.00      1.00      1.00        20
    positive       0.91      1.00      0.95        20

    accuracy                           0.97        60
   macro avg       0.97      0.97      0.97        60
weighted avg       0.97      0.97      0.97        60

[[18  0  2]
 [ 0 20  0]
 [ 0  0 20]]


### Report:
---

Based off of the topic modeling that was performed earlier, The main 6 topics that were constructed should give us a clear view of what our customers often discuss when using our product. Below is further analysis on each topic after combining them into the main dataset to further filter out which ones are in fact the ones that are the key pain points:

In [ ]:
TOPICS_DICT = {
    0:"Bike Comfort Quality",
    1:"Customer Service",
    2:"Ride Experience",
    3:"Bike Overall Quality",
    4:"Rental Experience",
    5:"Bike Battery Life",
    6:"Bike Condition"
}

topic_reviews = pd.DataFrame({
    "review_text":reviews_df["review_text"].tolist(),
    "sentiment":reviews_df["sentiment"].tolist(),
    "topic":topics
})

topic_reviews["topic"] = topic_reviews["topic"].astype(int).map(TOPICS_DICT)

filter = topic_reviews[topic_reviews["sentiment"] == "negative"]
filter.groupby("topic").count()

,review_text,sentiment
topic,,
Bike Battery Life,10,10
Bike Comfort Quality,1,1
Bike Condition,20,20
Bike Overall Quality,10,10
Customer Service,29,29
Rental Experience,10,10
Ride Experience,20,20


### Therefore, based on the data above, it looks like Ride Experience is the primary area of focus that needs to be fixed.

### Final Note.

As mentioned earlier, if we are going to perform more pertinent analyisis in the future, the datasets we work with are going to need to be much larger in order to obtain quality results. On total, there were 49700 duplicates located in this dataset out of 49999 rows which only leaves us with exactly 299 rows of data to work with. This is fine for training naive bayes models which are not data hungry, but should we use a transformer or LSTM, we definitely need to consider holding off until we have 49999 unique rows of reviews not 49999 reviews with 49700 duplicates.

below is a summary of the duplicated values for the review text column in the dataset used before the duplicate reviews were dropped:

In [ ]:
pre_review_df = pd.read_csv("bike_rental_reviews.csv")
pre_review_df["review_text"].value_counts()

,count
review_text,
Worst experience ever. The mobile app ruined the entire ride.,201
Not worth the money. The seat comfort was a major letdown.,200
Long wait times and the support staff was awful.,197
Worst experience ever. The bike condition ruined the entire ride.,197
Superb experience renting bikes. The mobile app was outstanding.,197
...,...
A decent ride but not the best rental station.,138
"The entire process was easy, and the battery life was high quality.",136
Not worth the money. The mobile app was a major letdown.,135
